## 在运行本笔记本之前，请先下载以下工具：

1. **查找当前的 Google Chrome 版本**  
   - 打开 Google Chrome，并在地址栏输入：  
     ```
     chrome://settings/help
     ```
   - 建议使用 **133** 版本的 Google Chrome（或最新可用的稳定版本）。

2. **下载对应版本的 ChromeDriver**  
   - 前往官方的 Chrome 测试下载页面：  
     [https://googlechromelabs.github.io/chrome-for-testing/#stable](https://googlechromelabs.github.io/chrome-for-testing/#stable)  
   - 下载与您的 Chrome 版本相匹配的 ChromeDriver。  
   - 例如，如果您的 Chrome 版本是 133，请下载 **ChromeDriver 133**。

3. **查找已下载的 ChromeDriver**  
   - 如果您使用 macOS，可在终端 (Terminal) 中运行以下命令来查找 `chromedriver` 的位置：
     ```bash
     mdfind -name chromedriver
     ```
   - 如果使用其他操作系统，请检查默认的 **下载** 目录或您保存该文件的目录。

In [26]:
# 导入必要的包
import os
import re
import time
import json
import random
import pymongo
import pandas as pd
import requests  # 新增：用于下载图片

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

##### 启动 Chrome 浏览器实例：

打开 **terminal**, 下载 Chrome Driver (假的 Google Chrome)

```bash
brew install chromedriver
chmod +x /opt/homebrew/bin/chromedriver
```

输入以下命令（将 `your Chrome.exe path` 替换为您的 Google Chrome 浏览器路径）：
```bash
<your Chrome.exe path> --remote-debugging-port=9222 --user-data-dir="/Users/<your home folder name>/selenium/AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)
- 来做个比方， 我的 *terminal command* 会是:

/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir="/Users/princess/selenium/AutomationProfile"

In [27]:
# 配置Chrome浏览器
options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
options.add_argument('--incognito')
options.add_argument("--headless")
browser = webdriver.Chrome(options=options)
action = ActionChains(browser)

In [28]:
key_word = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(1)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(1)
            break

def check_page_load_status(browser, keyword):
    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    
    while True:
        if keyword in browser.title:
            print('加载成功')
            print('检查时间:', time.ctime())
            break
        else:
            time.sleep(1)

def selenium_test():
    """
    登录状态检查，网页加载检查，根据用户输入进行搜索
    """
    global key_word, num
    browser.get('https://www.xiaohongshu.com/explore')
    
    check_login_status(browser)
    
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    keyword = input("搜索关键词：")
    key_word = keyword
    
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    url = f'https://www.xiaohongshu.com/search_result?keyword={keyword}&source=web_explore_feed'
    browser.get(url)
    time.sleep(1)
    
    check_page_load_status(browser, keyword)

selenium_test()

即将开始检查小红书登录状态...
爬取数据有账户封禁的风险，建议使用非主账号登录。
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:34 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:35 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:36 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:37 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:38 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:39 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:40 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:41 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:42 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:43 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:44 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:45 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:46 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:47 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:48 2025
暂未登录，请手动登录
检查时间: Wed Feb 12 19:57:49 2025
登录成功
检查时间: Wed Feb 12 19:57:50 2025
请在文本框中根据提示输入搜索关键词和笔记爬取数量。
即将开始检查网页加载状态...
如果网页进入人机验证页面，请先手动完成验证。
加载成功
检查时间: Wed Feb 12 19:58:01 2025


In [29]:
def change_mode(browser):
    # 更改模式为图文
    try:
        mode_button = browser.find_element(By.XPATH, '//*[@id="search-type"]/div/div/div[2]')
        mode_button.click()
        print('已自动更改模式为图文。')
    except Exception as e:
        print(f"更改模式失败: {e}")

selected_order_text = ''
def change_sort_order(browser, action):
    # 更改排序方式
    sort_order = {
        '综合': 1,
        '最新': 2,
        '最热': 3
    }
    print("请选择排序方式:")
    for idx, order in sort_order.items():
        print(f'{order}. {idx}')
    
    try:
        global selected_order_text
        selected_order_text = input("请输入排序方式对应的名称: ").strip()
        if selected_order_text not in sort_order:
            print("请输入有效的排序方式...")
            return
        
        selected_order_index = sort_order[selected_order_text]
    except Exception as e:
        print(f"处理排序选择时出错: {e}")
        return

    try:
        element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
        action.move_to_element(element).perform()# 模拟鼠标悬停
        menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
        option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
        option.click()# 模拟鼠标点击

        print('已选择排序方式为:',selected_order_text)
        print('检查时间:',time.ctime())

    except Exception as e:
        print(f"更改排序方式失败: {e}")

change_mode(browser)
change_sort_order(browser, action)

已自动更改模式为图文。
请选择排序方式:
1. 综合
2. 最新
3. 最热
请输入有效的排序方式...


In [30]:
# 初始化数据存储列表
authorName_list = []
likeNr_list = []
id_list = []
noteURL_list = []
userURL_list = []
commentNr_list = []
post_title_list = [] 
caption_list = []  
datePublished_list = []
images_list = []
author_avatar_list = [] 
starNr_list = []
authorCollectNr_list = []
authorFansNr_list = []
authorNoteNr_list = []
video_urls = [] 

def parsePage(page_source):
    """
    解析当前页面的HTML内容，提取笔记的基本信息并更新对应的列表。
    """
    response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
    selector = Selector(response)
    print("正在分析页面结构...")

    containers = ['//div[contains(@class, "note-item")]']
    for container in containers:
        elements = selector.xpath(container)
        if elements:
            print(f"找到容器，xpath: {container}")

    content_elements = selector.xpath('//section[contains(@class, "note-item")]')
    if content_elements:
        print(f"找到 {len(content_elements)} 个内容元素")
        for element in content_elements:
            try:
                # 提取URL
                note_url = element.xpath('.//a[contains(@class, "cover")]/@href').get()
                if note_url:
                    note_id = note_url.split('/')[-1].split('?')[0]
                    if note_id in id_list:
                        continue  # 避免重复
                    id_list.append(note_id)
                    noteURL_list.append(note_url)

                    # 提取作者名字
                    author = element.xpath('.//div[contains(@class, "author-wrapper")]//span[contains(@class, "name")]/text()').get()
                    authorName_list.append(author.strip() if author else "N/A")

                    # 提取点赞数量
                    likes = element.xpath('.//span[contains(@class, "count")]/text()').get()
                    likeNr_list.append(likes.strip() if likes else "0")

                    # 提取帖子标题
                    post_title = element.xpath('.//a[contains(@class, "title")]//span/text()').getall()
                    post_title_cleaned = ' '.join([c.strip() for c in post_title if c.strip()])
                    post_title_list.append(post_title_cleaned if post_title_cleaned else "N/A")

                    # 提取图片（主图）
                    main_image = element.xpath('.//a[contains(@class, "cover")]/img/@src').get()
                    images_list.append(main_image.strip() if main_image else "N/A")

                    # 提取头像图片
                    avatar_image = element.xpath('.//a[contains(@class, "author")]/img/@src').get()
                    author_avatar_list.append(avatar_image.strip() if avatar_image else "N/A")

                    # 提取用户URL
                    user_url = element.xpath('.//a[contains(@class, "author")]/@href').get()# 用户URL
                    userURL_list.append(user_url)

                    qbar.update(1)
            except Exception as e:
                print(f"处理元素时出错: {str(e)}")
                continue
    print(f"当前已爬取总数: {len(id_list)}")

def parseUserPage(user_url):
    """
    访问用户页面，提取附加字段，包括作者获赞/收藏数量、粉丝数量和笔记数量。
    同时初始化后续需要更新的 note-specific 字段：
        commentNr_list, datePublished_list, starNr_list, video_urls, caption_list
    """
    try:
        full_user_url = f'https://www.xiaohongshu.com{user_url}'
        browser.get(full_user_url)

        # 模拟滚动页面直到加载完成
        while True:
            previous_page_height = browser.execute_script("return document.body.scrollHeight")
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# 滚动到页面底部
            time.sleep(random.uniform(1, 2))

            current_page_height = browser.execute_script("return document.body.scrollHeight")
            if current_page_height == previous_page_height:
                break
            previous_page_height = current_page_height

        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="userPostedFeeds"]//section')))# 等待页面加载完成
        html = browser.page_source
        selector = Selector(text=html)
        
        # 提取作者的获赞/收藏数量
        author_collect_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[3]/span[@class="count"]/text()').extract_first()# 作者获赞与收藏数量
        author_collect_nr = author_collect_nr.strip() if author_collect_nr else "0"

        # 提取作者的粉丝数量
        author_fans_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[2]/span[@class="count"]/text()').extract_first()# 作者粉丝数量
        author_fans_nr = author_fans_nr.strip() if author_fans_nr else "0"

        # 提取作者的笔记数量（基于页面中 note 项的数量）
        author_note_nr = len(selector.xpath('//*[@id="userPostedFeeds"]//section'))# 作者笔记数量
        author_note_nr = str(author_note_nr)

        # 更新全局列表
        authorCollectNr_list.append(author_collect_nr)
        authorFansNr_list.append(author_fans_nr)
        authorNoteNr_list.append(author_note_nr)

        # 初始化后续待更新的 note‑specific 字段
        commentNr_list.append("0")
        datePublished_list.append("N/A")
        starNr_list.append("0")
        video_urls.append("N/A")
        caption_list.append("N/A")
        
        print("已提取作者附加字段。")
    except Exception as e:
        print(f"提取作者附加字段时出错, 错误: {str(e)}")

def parseNotePage(note_url, note_id):
    """
    访问每个笔记的页面，提取附加字段，包括评论数量、发布时间、收藏数量、粉丝数量、笔记数量、视频URL以及帖子内容（Caption）。
    """
    try:
        full_note_url = f'https://www.xiaohongshu.com{note_url}'
        browser.get(full_note_url)
        
        # 显式等待meta标签加载完成
        wait = WebDriverWait(browser, 10)
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))
        
        page_source = browser.page_source
        response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
        selector = Selector(response)

        # 提取各字段
        comments = selector.xpath('//*[@class="total"]/text()').get()
        comments = comments.strip() if comments else "0"

        date_published = selector.xpath('//*[@class="date"]/text()').get()
        if not date_published:
            date_published = selector.xpath('//time/@datetime').get()
        date_published = date_published.strip() if date_published else "N/A"

        stars = selector.xpath('//*[@class="count"]/text()').get()
        stars = stars.strip() if stars else "0"

        collect_nr = selector.xpath('//span[contains(@class, "collect") or contains(@class, "saved")]/text()').get()
        collect_nr = collect_nr.strip() if collect_nr else "0"

        fans_nr = selector.xpath('//span[contains(@class, "fans") or contains(@class, "followers")]/text()').get()
        fans_nr = fans_nr.strip() if fans_nr else "0"

        note_nr = selector.xpath('//span[contains(@class, "notes") or contains(@class, "posts")]/text()').get()
        note_nr = note_nr.strip() if note_nr else "0"

        video_url = selector.xpath('//video/@src').get()
        video_url = video_url.strip() if video_url else "N/A"

        caption = selector.xpath('//*[@name="description"]/@content').get()
        caption = caption.strip() if caption else "N/A"

        # 更新全局列表
        if note_id in id_list:
            index = id_list.index(note_id)
            commentNr_list[index] = comments
            datePublished_list[index] = date_published
            starNr_list[index] = stars
            authorCollectNr_list[index] = collect_nr
            authorFansNr_list[index] = fans_nr
            authorNoteNr_list[index] = note_nr
            video_urls[index] = video_url
            caption_list[index] = caption
            print(f"已提取附加字段，笔记ID: {note_id}")
        else:
            print(f"笔记ID {note_id} 未在id_list中找到。")
    except Exception as e:
        print(f"提取附加字段时出错，笔记ID: {note_id}, 错误: {str(e)}")

def ensure_search_results():
    """
    确保已导航到搜索结果页面，并选择“图文”模式。
    """
    current_url = browser.current_url
    print(f"当前URL: {current_url}")

    search_url = f'https://www.xiaohongshu.com/search_result?keyword={key_word}&source=web_explore_feed'
    browser.get(search_url)

    try:
        wait = WebDriverWait(browser, 10)
        wait.until(EC.title_contains(key_word))
        print("已导航到搜索结果页面。")
    except:
        print("导航到搜索结果页面时超时。")
        browser.quit()
        exit()

    try:
        selectors = [
            "//div[text()='图文']",
            "//div[contains(@class, 'tab')]//span[text()='图文']",
            "//div[contains(@class, 'filter')]//div[text()='图文']",
            "//*[contains(text(), '图文')]"
        ]
        for sel in selectors:
            try:
                element = WebDriverWait(browser, 10).until(
                    EC.element_to_be_clickable((By.XPATH, sel))
                )
                print(f"找到元素，选择器: {sel}")
                element.click()
                time.sleep(1)
                break
            except:
                continue
        else:
            print("未找到“图文”标签，可能页面结构已更改。")
    except Exception as e:
        print(f"切换视图时出错: {e}")

    time.sleep(1)

def download_image(url, save_path):
    """
    下载图片并保存到指定路径。
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"下载图片时出错，URL: {url}, 错误: {str(e)}")
        return False

def create_directories():
    """
    创建用于存储主图片和头像图片的目录。
    """
    if not os.path.exists('images_keyword'):
        os.makedirs('images_keyword')
    if not os.path.exists('avatars_keyword'):
        os.makedirs('avatars_keyword')

# 创建图片保存目录
create_directories()

# 定义进度条：用于实时跟踪已爬取的笔记数量
qbar = tqdm(total=num, desc="爬取进度")

ensure_search_results()

while len(id_list) < num:
    for _ in range(3):
        browser.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)
    parsePage(browser.page_source)
    if '- THE END -' in browser.page_source or 'No more content' in browser.page_source:
        print(f"已到达内容末尾。总共收集: {len(id_list)} 条")
        break
    time.sleep(random.uniform(0.5, 1))

print(f"总共收集的条目数: {len(id_list)}")
if id_list:
    print("收集的数据样本:")
    for i in range(min(5, len(id_list))):
        print(f"URL: {id_list[i]}")

if len(id_list) > num:
    id_list = id_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]
    noteURL_list = noteURL_list[:num]  
    commentNr_list = commentNr_list[:num]
    post_title_list = post_title_list[:num]
    datePublished_list = datePublished_list[:num]
    images_list = images_list[:num]
    author_avatar_list = author_avatar_list[:num]
    starNr_list = starNr_list[:num]
    authorCollectNr_list = authorCollectNr_list[:num]
    authorFansNr_list = authorFansNr_list[:num]
    authorNoteNr_list = authorNoteNr_list[:num]
    video_urls = video_urls[:num]
    caption_list = caption_list[:num]

print(f"截断后的总条目数: {len(id_list)}")
print("收集的数据样本:")
for i in range(min(5, len(id_list))):
    print(f"作者: {authorName_list[i]}, 点赞: {likeNr_list[i]}, URL: {id_list[i]}")
qbar.close()

# 提取附加字段（帖子内容、日期发布、评论数量等）
print("开始提取用户信息，包括收藏数量、粉丝数量和笔记数量...")
qbar = tqdm(total=len(userURL_list), desc="已提取的用户信息")
for user_url in userURL_list:
    parseUserPage(user_url)
    qbar.update(1)
    time.sleep(random.uniform(0.5, 1))
qbar.close()

print("开始提取附加字段，包括帖子内容、日期发布和评论数量...")
qbar = tqdm(total=len(id_list), desc="已获取的笔记数量...")
for note_id, note_url in zip(id_list, noteURL_list):
    parseNotePage(note_url, note_id)
    qbar.update(1)
    time.sleep(random.uniform(0.5, 1))
qbar.close()

# 下载图片
print("开始下载主图片和头像图片...")

# 下载主图片
image_download_bar = tqdm(total=len(images_list), desc="下载主图片")
for idx, image_url in enumerate(images_list):
    if image_url == "N/A":
        image_download_bar.update(1)
        continue
    image_extension = os.path.splitext(image_url)[1].split('?')[0]
    if image_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        image_extension = '.jpg'
    image_filename = f"images_keyword/{id_list[idx]}{image_extension}"
    success = download_image(image_url, image_filename)
    if not success:
        image_filename = "N/A"
    images_list[idx] = image_filename
    image_download_bar.update(1)
image_download_bar.close()

# 下载头像图片
avatar_download_bar = tqdm(total=len(author_avatar_list), desc="下载头像图片")
for idx, avatar_url in enumerate(author_avatar_list):
    if avatar_url == "N/A":
        avatar_download_bar.update(1)
        continue
    avatar_extension = os.path.splitext(avatar_url)[1].split('?')[0]
    if avatar_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        avatar_extension = '.jpg'
    avatar_filename = f"avatars_keyword/{id_list[idx]}{avatar_extension}"
    success = download_image(avatar_url, avatar_filename)
    if not success:
        avatar_filename = "N/A"
    author_avatar_list[idx] = avatar_filename
    avatar_download_bar.update(1)
avatar_download_bar.close()

print("所有图片下载完成。")

# 创建数据字典
data = {
    'Author Name': authorName_list,
    'Likes': likeNr_list,
    'Comments': commentNr_list,
    'Post Title': post_title_list, 
    'Caption': caption_list,
    'Date Published': datePublished_list,
    'Images': images_list,
    'Author Avatar': author_avatar_list,
    'Stars': starNr_list,
    'Author Collect Nr': authorCollectNr_list,
    'Author Fans Nr': authorFansNr_list,
    'Author Note Nr': authorNoteNr_list,
    'Video URL': video_urls,
    'Post ID': id_list,
    'User URL': userURL_list,
    'Note URL': noteURL_list
}

df = pd.DataFrame(data)
df.set_index('Post ID', inplace=True)
print(df.head())
df.to_csv('scraped_xhs_keyword.csv', encoding='utf-8-sig')
print("数据已保存到 'scraped_xhs_keyword.csv'")

爬取进度:   0%|          | 0/10 [00:00<?, ?it/s]

当前URL: https://www.xiaohongshu.com/search_result?keyword=%E7%8B%97%E7%B2%AE&source=web_explore_feed&type=51
已导航到搜索结果页面。
找到元素，选择器: //*[contains(text(), '图文')]
正在分析页面结构...
找到容器，xpath: //div[contains(@class, "note-item")]
找到 18 个内容元素
当前已爬取总数: 16
总共收集的条目数: 16
收集的数据样本:
URL: 67a9ba5a000000001902e37f
URL: 67ac08b6000000001703ab73
URL: 673845cc000000001b011357
URL: 666fef14000000000e032cb5
URL: 640c28810000000013012bb8
截断后的总条目数: 10
收集的数据样本:
作者: 祁花椒, 点赞: 6, URL: 67a9ba5a000000001902e37f
作者: 还好老子可爱, 点赞: 2, URL: 67ac08b6000000001703ab73
作者: 不冷狗狗, 点赞: 121, URL: 673845cc000000001b011357
作者: Hello Pets Plus 宠物日用, 点赞: 78, URL: 666fef14000000000e032cb5
作者: 柴布丁日常, 点赞: 565, URL: 640c28810000000013012bb8
开始提取用户信息，包括收藏数量、粉丝数量和笔记数量...


已提取的用户信息:   0%|          | 0/10 [00:00<?, ?it/s]

已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
已提取作者附加字段。
开始提取附加字段，包括帖子内容、日期发布和评论数量...


已获取的笔记数量...:   0%|          | 0/10 [00:00<?, ?it/s]

已提取附加字段，笔记ID: 67a9ba5a000000001902e37f
已提取附加字段，笔记ID: 67ac08b6000000001703ab73
已提取附加字段，笔记ID: 673845cc000000001b011357
已提取附加字段，笔记ID: 666fef14000000000e032cb5
已提取附加字段，笔记ID: 640c28810000000013012bb8
已提取附加字段，笔记ID: 64982849000000001300ab83
已提取附加字段，笔记ID: 66978f81000000000d00d491
已提取附加字段，笔记ID: 667bcd97000000001f004b81
已提取附加字段，笔记ID: 6662df6d000000001303ec3a
已提取附加字段，笔记ID: 66b9db5e0000000009017f85
开始下载主图片和头像图片...


下载主图片:   0%|          | 0/10 [00:00<?, ?it/s]

下载头像图片:   0%|          | 0/10 [00:00<?, ?it/s]

所有图片下载完成。
                                   Author Name Likes Comments  \
Post ID                                                         
67a9ba5a000000001902e37f                   祁花椒     6        0   
67ac08b6000000001703ab73                还好老子可爱     2        0   
673845cc000000001b011357                  不冷狗狗   121        0   
666fef14000000000e032cb5  Hello Pets Plus 宠物日用    78        0   
640c28810000000013012bb8                 柴布丁日常   565        0   

                                      Post Title          Caption  \
Post ID                                                             
67a9ba5a000000001902e37f       喂了很久这几款狗粮，觉得效果很夸张  3 亿人的生活经验，都在小红书   
67ac08b6000000001703ab73      不同品种中小型犬狗粮推荐 | 实惠篇  3 亿人的生活经验，都在小红书   
673845cc000000001b011357      真心建议！幼犬粮都按这个标准挑选！！  3 亿人的生活经验，都在小红书   
666fef14000000000e032cb5  🇲🇾鱼肉犬粮 | 不再头疼选狗粮，养狗人必看  3 亿人的生活经验，都在小红书   
640c28810000000013012bb8       亲测 | 前🔟进口狗粮品牌测评推荐  3 亿人的生活经验，都在小红书   

                         Date Published  \
Post ID